In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.yaml')
w


{'email_address': 'brainbummer@mailinator.com',
 'name': 'test',
 'products':       description       stock_code
 0  organic cheese           281739
 1    GB chocolate           260803
 2            fake  fake_stock_code}

In [3]:
# Get and parse a product

code = w['products']['stock_code'].iat[-1]
response = cd.get_product(code)
cd.parse_product(response)

OrderedDict([('stock_code', 'fake_stock_code'),
             ('name', None),
             ('description', None),
             ('size', None),
             ('sale_price', None),
             ('price', None),
             ('discount_percentage', None),
             ('unit_price', None),
             ('datetime', '2017-06-04T19:40:20')])

In [4]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=False)
f

CPU times: user 288 ms, sys: 8 ms, total: 296 ms
Wall time: 1.27 s


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,7.3,11.00,33.636364,$14.60/1KG,2017-06-04 19:40:20
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,3.5,3.89,10.025707,$3.50/100G,2017-06-04 19:40:21
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-06-04 19:40:21


In [5]:
# Get many products asynchronously and parse

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=True)
f

CPU times: user 264 ms, sys: 0 ns, total: 264 ms
Wall time: 1.02 s


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,7.3,11.00,33.636364,$14.60/1KG,2017-06-04 19:40:22
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,3.5,3.89,10.025707,$3.50/100G,2017-06-04 19:40:22
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-06-04 19:40:22


In [6]:
# Filter sales

g = cd.filter_sales(f)
g

,name,sale_price,price,discount_percentage
0,Mainland Cheese Block Organic Cheddar 500g,7.3,11.00,33.636364
1,Green & Blacks Chocolate Block Organic Dark Ch...,3.5,3.89,10.025707


In [7]:
# Email sales

domain = cd.get_secret('mailgun_domain')  # replace with your Mailgun domain
key = cd.get_secret('mailgun_api_key')  # replace with your Mailgun API key
cd.email(g, 'brainbummer@mailinator.com', domain, key)

<Response [200]>

In [8]:
# Run pipeline: read watchlist, collect products, write products, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.yaml', OUT_DIR, domain, key)